In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import spacy
import re
import os
import nltk

nlp = spacy.load('en')
doc = nlp("I cured the illness. The illness was cured by me. A red cup. I ran quickly.")
doc_list = list(doc)
def flatten_subtree(st):
    return ''.join([w.text_with_ws for w in list(st)]).strip()

#### Create placeholders for results

In [2]:
clin_concepts_set = []
clin_classes_set_prelim  = []

### Aggregate mentions of clinical concept (i.e. 1 number per consultation)
clin_concepts_number_per_consultation = {}

### Aggregate mentions of each clinical classification (separately) (i.e. >1 number per consultation)
clin_classes_type_number_per_consultation = {}

#### Create a dictionary linking each clinical concept to its classification

In [3]:

opened = open('defaultDict.txt', "r")
print(opened)
opened
lines = opened.readlines()

## the clinical concept (e.g. blood) = always appears before \t
##      this can be multiple words separated by spaces
## the classification (e.g. TREATMENT) = always appears after \t
##      this can be multiple words separated by hyphens (I.E. if multiple classes apply to the same concept)
"""
EXAMPLES:
Clip	THERPROCDEV
CT	PROBLEM-THERPROCDEV-DIAGPROC-LABTEST
CHEST	BODYLOC
Reason	BODYMEAS
Radiology	DIAGPROC-LABTEST
carotid artery bypass	THERPROCDEV
"""


clin_concepts_dict = {}

for x in lines:
    concept          =  (re.split(r'\t+', x))[0]
    classification   =  (re.split(r'\t+', x))[1]
    clin_concepts_dict[concept] = classification

###  remove "/n" from the value
clin_concepts_dict = dict(map(str.strip,x) for x in clin_concepts_dict.items()) 
###  lower case - apply to key.
clin_concepts_dict =  {k.lower(): v for k, v in clin_concepts_dict.items()}

clin_concepts_dict

<_io.TextIOWrapper name='defaultDict.txt' mode='r' encoding='cp1252'>


<_io.TextIOWrapper name='defaultDict.txt' mode='r' encoding='cp1252'>

'\nEXAMPLES:\nClip\tTHERPROCDEV\nCT\tPROBLEM-THERPROCDEV-DIAGPROC-LABTEST\nCHEST\tBODYLOC\nReason\tBODYMEAS\nRadiology\tDIAGPROC-LABTEST\ncarotid artery bypass\tTHERPROCDEV\n'

{'clip': 'THERPROCDEV',
 'ct': 'PROBLEM-THERPROCDEV-DIAGPROC-LABTEST',
 'chest': 'BODYLOC',
 'reason': 'BODYMEAS',
 'radiology': 'DIAGPROC-LABTEST',
 'last name': 'BODYMEAS',
 'ap': 'TOD-PROBLEM-THERPROCDEV',
 'examination': 'PROBLEM',
 'condition': 'BODYMEAS',
 'evidence': 'THERPROCDEV',
 'admitting diagnosis': 'PROBLEM',
 'indication': 'THERPROCDEV',
 'catheter': 'THERPROCDEV',
 'well': 'DISORDER-PROBLEM',
 'study': 'LABTEST',
 '%': 'UNIT',
 'day': 'TUNIT',
 'procedure': 'THERPROCDEV-DIAGPROC-LABTEST',
 'tablet': 'DDF-DOSEUNIT-BIOSUB',
 'dr .': 'PROBLEM',
 'pneumothorax': 'DISORDER-PROBLEM',
 'procedural': 'THERPROCDEV-DIAGPROC-LABTEST',
 'cath': 'THERPROCDEV',
 'position': 'THERPROCDEV',
 'tube': 'THERPROCDEV',
 'findings': 'DISORDER-PROBLEM',
 'sheath': 'THERPROCDEV',
 'po': 'RUT-PROBLEM',
 'unchanged': 'PROBLEM',
 'over': 'DISORDER-PROBLEM',
 'fluid': 'THERPROCDEV',
 'advanced': 'THERPROCDEV',
 'first name': 'BODYMEAS',
 'likely': 'BODYMEAS',
 'atelectasis': 'DISORDER-PROBLEM',
 '

In [4]:
not_clinical_term = ["aaa", 
    "ah", 
    "all", 
    "am", 
    "an", 
    "and", 
    "as", 
    "ask", 
    "asked", 
    "asking", 
    "at", 
    "be ", 
    "before", 
    "but", 
    "call", 
    "can", 
    "chat", 
    "chips", 
    "cop", 
    "day", 
    "days", 
    "did", 
    "do", 
    "does", 
    "doing", 
    "don", 
    "door", 
    "down", 
    "due", 
    "e", 
    "eat", 
    "eating", 
    "eh", 
    "eight", 
    "eleven", 
    "em", 
    "even", 
    "ever", 
    "feel", 
    "feeling", 
    "feels", 
    "find", 
    "finding", 
    "five", 
    "food", 
    "for", 
    "four", 
    "get", 
    "getting", 
    "go", 
    "going", 
    "got", 
    "h", 
    "had", 
    "hang", 
    "hard", 
    "he", 
    "her", 
    "hi", 
    "hm", 
    "home", 
    "hundred", 
    "if", 
    "initially", 
    "into", 
    "is", 
    "isn", 
    "laughs", 
    "leaning", 
    "leave", 
    "let", 
    "like", 
    "listen", 
    "literally", 
    "little", 
    "ll", 
    "look", 
    "looking", 
    "lovely", 
    "me", 
    "meet", 
    "mic", 
    "mind", 
    "mm", 
    "morning", 
    "mrs", 
    "n1", 
    "name", 
    "ninety", 
    "no", 
    "not", 
    "oh", 
    "okay", 
    "on", 
    "one", 
    "organised", 
    "other", 
    "questions", 
    "re", 
    "run", 
    "said", 
    "see", 
    "set", 
    "seven", 
    "since", 
    "sit", 
    "sitting", 
    "six", 
    "sixty", 
    "smoke", 
    "sort", 
    "sorted", 
    "stay", 
    "still", 
    "switch", 
    "talk", 
    "ten", 
    "think", 
    "thirty", 
    "this", 
    "three", 
    "too", 
    "transferred", 
    "turn", 
    "twelve", 
    "twenty", 
    "two", 
    "understand", 
    "until", 
    "us", 
    "used", 
    "ve", 
    "want", 
    "was", 
    "well", 
    "while", 
    "x1", 
    "x2", 
    "year"]


In [5]:
# XXX drop key from clin_concepts_dict  if  NOT IN not_clinical_term


### Prepare the tokeniser

In [6]:
%run -i sentiment_tokenize.py


### Create list of all classes (for dictionary of count of classes)

In [7]:
list_classes = ['BIOSUB',
    'BODYLOC',
    'BODYMEAS',
    'DDF',
    'DIAGPROC',
    'DISORDER',
    'DOSEUNIT',
    'DRT',
    'DRUG',
    'FREQ',
    'LABTEST',
    'NUM',
    'PROBLEM',
    'THERPROCDEV',
    'TK',
    'TOD',
    'TUNIT',
    'UNIT']
    

### START LOOP HERE - OVER EACH TRANSCRIPT SEPARATELY

In [8]:
counter = 0
folder  = "sentiment_text_data"

number_of_files = len(os.listdir(folder))
print("Number of files in the directory: {}".format(number_of_files))

while counter < number_of_files:
    
    ### print file name and number:
    file = os.listdir(folder)[counter]
    print()
    print("File number: {}".format(counter + 1))
    print("File name: {}".format(file))
    print()
    
    ### tokenise
    tokens = open_and_tokenize(counter, folder)
    type(tokens)
    
    ### Dictionary to count no. times each class occurs.(will insert into "clin_classes_type_number_per_consultation" for every consultation)
    count_each_class_type = {}
    for x in list_classes:
        count_each_class_type[x] = 0
    
    # XXX - just so it runs quickly during development.
    my_text_strings = tokens[0:400]
    
    ### placeholders for results (separate for each consultation)
    classifications_length = 0
    concepts_length        = 0
    all_classifications_consultation_prelim = []

    ### counter for the "for" loop over each token below
    token_number = 1

    ### loop over each token
    for mystring in my_text_strings:
        
        ### store the class & concept (if the token is a "clinical concept")
        classifications_consultation = [value for key, value in clin_concepts_dict.items() if mystring == key.lower()]
        concepts_consultation        = [key for key, value in clin_concepts_dict.items() if mystring == key.lower()]       

        ### add to the master set of concepts / classes across all transcripts.
        clin_classes_set_prelim.extend(classifications_consultation)    
        clin_concepts_set.extend(concepts_consultation)
            # by using "extend", each item added is a STRING. if use "append", an item added at the end 
            # would be a full list (could not use "set" on it)

        ### update the number of clinical concepts per consultation
        classifications_length += len(classifications_consultation)
        concepts_length        += len(concepts_consultation)
        
        ### add the "classification" (e.g. treatment / problem) to the list of classifications for the specific transcript
        if classifications_consultation !=  []:
            all_classifications_consultation_prelim.extend(classifications_consultation)
        
        ### just a simple progress indicator for the token loop
        if token_number % 200 == 0:
            print("Progress tracking: File number {}, Token number {} (of a total of {} tokens)".format(counter+1, token_number, len(tokens)))
        
        ### and back to start of token loop
        token_number += 1
    
    
    print("")
    print("all_classifications_consultation_prelim: before split on hyphen")
    print(all_classifications_consultation_prelim[0:50])

    ### some tokens have multiple classes (e.g. "treatment-problem") - split on the hyphen to account for each class separately.
    all_classifications_consultation_prelim  = [i.split('-', 10)[:] for i in all_classifications_consultation_prelim]

    ### check that the split occurred ok and nothing is missing.
    print("")
    print("all_classifications_consultation_prelim: after split on hyphen")
    all_classifications_consultation_prelim[0:50]

    ### the final container for the classes for a consultation (i.e. put it in the right List form)
    all_classifications_consultation_final = []
    for x in all_classifications_consultation_prelim:
        all_classifications_consultation_final.extend(x)
    ### just show the first 20 (this confirms the content and the appropriate object type)
    all_classifications_consultation_final[0:20]

    ### put info from list into Dictionary format (for this particular consultation)
    for x in all_classifications_consultation_final:
        count_each_class_type[x] +=1 
    print()
    print("count_each_class_type: i.e. the dictionary containing the number of each class for a consultation")
    count_each_class_type
        
        
    print()
    print("classifications_length: for one transcript")
    classifications_length
    print()
    print("concepts_length: for one transcript")
    concepts_length
    
        ## number of times any clinical concept mentioned: add to master dictionary for all transcripts
    clin_concepts_number_per_consultation[str(counter+1)] = concepts_length
        ## number of times each type of clinical classification mentioned: add to master dictionary for all transcripts
    clin_classes_type_number_per_consultation[str(counter+1)] = count_each_class_type
    
    counter  +=  1


Number of files in the directory: 8

File number: 1
File name: 1_1 PR.txt


File number: 1
File name: 1_1 PR.txt
Open file and display 'type' = <class '_io.TextIOWrapper'>
Read file and display 'type' = <class 'str'>
Tokenise file and display 'type' = <class 'list'>

Just show the first & last 10 words of the main dialogue text file
...These are first 10 words:
scenario
pos
aaa
d1
it
s
going
a
little
bit
...These are last 10 words:
pr43
pr44
pr45
pr46
pr47
pr48
pr49
pr50
pr51
1_1.mp3
return tokens


list

Progress tracking: File number 1, Token number 200 (of a total of 4012 tokens)
Progress tracking: File number 1, Token number 400 (of a total of 4012 tokens)

all_classifications_consultation_prelim: before split on hyphen
['DISORDER-PROBLEM-BIOSUB', 'DISORDER-PROBLEM', 'DISORDER-PROBLEM', 'PROBLEM', 'DOSEUNIT', 'BODYMEAS-DIAGPROC-LABTEST', 'BODYLOC', 'BODYLOC', 'TK-DISORDER-PROBLEM', 'TK-DISORDER-PROBLEM', 'TK-DISORDER-PROBLEM', 'BODYMEAS', 'TK-PROBLEM-LABTEST', 'DRUG', 'BODYMEAS', 'PROBLEM', 'PROBLEM-BODYMEAS', 'PROBLEM', 'PROBLEM', 'PROBLEM-BODYMEAS', 'TK-PROBLEM', 'NUM', 'NUM', 'NUM', 'DISORDER-PROBLEM', 'TK-PROBLEM-LABTEST', 'BODYMEAS', 'TK-PROBLEM', 'PROBLEM', 'TK-PROBLEM', 'TK-PROBLEM', 'PROBLEM', 'PROBLEM', 'BODYLOC', 'NUM', 'DRUG', 'DISORDER-PROBLEM', 'TK-PROBLEM', 'TK-PROBLEM', 'DISORDER', 'PROBLEM', 'NUM', 'NUM-BODYMEAS', 'TK-PROBLEM', 'PROBLEM', 'PROBLEM', 'TK-PROBLEM', 'NUM', 'DRUG-DIAGPROC-LABTEST', 'TK-PROBLEM-LABTEST']

all_classifications_consultation_prelim: after spl

[['DISORDER', 'PROBLEM', 'BIOSUB'],
 ['DISORDER', 'PROBLEM'],
 ['DISORDER', 'PROBLEM'],
 ['PROBLEM'],
 ['DOSEUNIT'],
 ['BODYMEAS', 'DIAGPROC', 'LABTEST'],
 ['BODYLOC'],
 ['BODYLOC'],
 ['TK', 'DISORDER', 'PROBLEM'],
 ['TK', 'DISORDER', 'PROBLEM'],
 ['TK', 'DISORDER', 'PROBLEM'],
 ['BODYMEAS'],
 ['TK', 'PROBLEM', 'LABTEST'],
 ['DRUG'],
 ['BODYMEAS'],
 ['PROBLEM'],
 ['PROBLEM', 'BODYMEAS'],
 ['PROBLEM'],
 ['PROBLEM'],
 ['PROBLEM', 'BODYMEAS'],
 ['TK', 'PROBLEM'],
 ['NUM'],
 ['NUM'],
 ['NUM'],
 ['DISORDER', 'PROBLEM'],
 ['TK', 'PROBLEM', 'LABTEST'],
 ['BODYMEAS'],
 ['TK', 'PROBLEM'],
 ['PROBLEM'],
 ['TK', 'PROBLEM'],
 ['TK', 'PROBLEM'],
 ['PROBLEM'],
 ['PROBLEM'],
 ['BODYLOC'],
 ['NUM'],
 ['DRUG'],
 ['DISORDER', 'PROBLEM'],
 ['TK', 'PROBLEM'],
 ['TK', 'PROBLEM'],
 ['DISORDER'],
 ['PROBLEM'],
 ['NUM'],
 ['NUM', 'BODYMEAS'],
 ['TK', 'PROBLEM'],
 ['PROBLEM'],
 ['PROBLEM'],
 ['TK', 'PROBLEM'],
 ['NUM'],
 ['DRUG', 'DIAGPROC', 'LABTEST'],
 ['TK', 'PROBLEM', 'LABTEST']]

['DISORDER',
 'PROBLEM',
 'BIOSUB',
 'DISORDER',
 'PROBLEM',
 'DISORDER',
 'PROBLEM',
 'PROBLEM',
 'DOSEUNIT',
 'BODYMEAS',
 'DIAGPROC',
 'LABTEST',
 'BODYLOC',
 'BODYLOC',
 'TK',
 'DISORDER',
 'PROBLEM',
 'TK',
 'DISORDER',
 'PROBLEM']

...

{'BIOSUB': 2,
 'BODYLOC': 8,
 'BODYMEAS': 9,
 'DDF': 0,
 'DIAGPROC': 0,
 'DISORDER': 33,
 'DOSEUNIT': 5,
 'DRT': 5,
 'DRUG': 10,
 'FREQ': 0,
 'LABTEST': 7,
 'NUM': 3,
 'PROBLEM': 103,
 'THERPROCDEV': 10,
 'TK': 42,
 'TOD': 1,
 'TUNIT': 1,
 'UNIT': 0}


classifications_length: for one transcript


146


concepts_length: for one transcript


146

### After loop -  look at results.

In [10]:
type(clin_classes_set_prelim)

### get rid of duplicates
clin_classes_set_prelim = set(clin_classes_set_prelim)

### put in format for splitting on hyphens (i.e. some tokens have multiple classes attached - account for them separately)
clin_classes_set_prelim = list(clin_classes_set_prelim)
print("")
print("Preliminary")
clin_classes_set_prelim

clin_classes_set_prelim = [i.split('-', 10)[:] for i in clin_classes_set_prelim]

clin_classes_set_final  = []
for x in clin_classes_set_prelim:
    clin_classes_set_final.extend(x)

print("")
print("Final")
clin_classes_set_final

list


Preliminary


['DISORDER-PROBLEM-BIOSUB',
 'DISORDER-PROBLEM',
 'DISORDER-PROBLEM',
 'PROBLEM',
 'DOSEUNIT',
 'BODYMEAS-DIAGPROC-LABTEST',
 'BODYLOC',
 'BODYLOC',
 'TK-DISORDER-PROBLEM',
 'TK-DISORDER-PROBLEM',
 'TK-DISORDER-PROBLEM',
 'BODYMEAS',
 'TK-PROBLEM-LABTEST',
 'DRUG',
 'BODYMEAS',
 'PROBLEM',
 'PROBLEM-BODYMEAS',
 'PROBLEM',
 'PROBLEM',
 'PROBLEM-BODYMEAS',
 'TK-PROBLEM',
 'NUM',
 'NUM',
 'NUM',
 'DISORDER-PROBLEM',
 'TK-PROBLEM-LABTEST',
 'BODYMEAS',
 'TK-PROBLEM',
 'PROBLEM',
 'TK-PROBLEM',
 'TK-PROBLEM',
 'PROBLEM',
 'PROBLEM',
 'BODYLOC',
 'NUM',
 'DRUG',
 'DISORDER-PROBLEM',
 'TK-PROBLEM',
 'TK-PROBLEM',
 'DISORDER',
 'PROBLEM',
 'NUM',
 'NUM-BODYMEAS',
 'TK-PROBLEM',
 'PROBLEM',
 'PROBLEM',
 'TK-PROBLEM',
 'NUM',
 'DRUG-DIAGPROC-LABTEST',
 'TK-PROBLEM-LABTEST',
 'DRUG',
 'BODYMEAS',
 'PROBLEM',
 'NUM',
 'DRUG-DIAGPROC-LABTEST',
 'TK-DISORDER-PROBLEM-BODYLOC',
 'TK-PROBLEM',
 'DRUG-DIAGPROC-LABTEST',
 'PROBLEM',
 'TK-DISORDER-PROBLEM-BODYLOC',
 'PROBLEM',
 'TK-PROBLEM',
 'TK-THERPROC


Final


['DISORDER',
 'PROBLEM',
 'BIOSUB',
 'DISORDER',
 'PROBLEM',
 'DISORDER',
 'PROBLEM',
 'PROBLEM',
 'DOSEUNIT',
 'BODYMEAS',
 'DIAGPROC',
 'LABTEST',
 'BODYLOC',
 'BODYLOC',
 'TK',
 'DISORDER',
 'PROBLEM',
 'TK',
 'DISORDER',
 'PROBLEM',
 'TK',
 'DISORDER',
 'PROBLEM',
 'BODYMEAS',
 'TK',
 'PROBLEM',
 'LABTEST',
 'DRUG',
 'BODYMEAS',
 'PROBLEM',
 'PROBLEM',
 'BODYMEAS',
 'PROBLEM',
 'PROBLEM',
 'PROBLEM',
 'BODYMEAS',
 'TK',
 'PROBLEM',
 'NUM',
 'NUM',
 'NUM',
 'DISORDER',
 'PROBLEM',
 'TK',
 'PROBLEM',
 'LABTEST',
 'BODYMEAS',
 'TK',
 'PROBLEM',
 'PROBLEM',
 'TK',
 'PROBLEM',
 'TK',
 'PROBLEM',
 'PROBLEM',
 'PROBLEM',
 'BODYLOC',
 'NUM',
 'DRUG',
 'DISORDER',
 'PROBLEM',
 'TK',
 'PROBLEM',
 'TK',
 'PROBLEM',
 'DISORDER',
 'PROBLEM',
 'NUM',
 'NUM',
 'BODYMEAS',
 'TK',
 'PROBLEM',
 'PROBLEM',
 'PROBLEM',
 'TK',
 'PROBLEM',
 'NUM',
 'DRUG',
 'DIAGPROC',
 'LABTEST',
 'TK',
 'PROBLEM',
 'LABTEST',
 'DRUG',
 'BODYMEAS',
 'PROBLEM',
 'NUM',
 'DRUG',
 'DIAGPROC',
 'LABTEST',
 'TK',
 'DISORDER

In [14]:
### set
clin_classes_set_final  =  set(clin_classes_set_final)
clin_concepts_set =  set(clin_concepts_set)

print("clin_classes_set")
clin_classes_set_final

print("Number of clinical concepts per consultation")
clin_concepts_number_per_consultation

print("How many times each classification appeared in each transcript")
clin_classes_type_number_per_consultation

print("clin_concepts_set")
clin_concepts_set
print()

clin_classes_set


{'BIOSUB',
 'BODYLOC',
 'BODYMEAS',
 'DDF',
 'DIAGPROC',
 'DISORDER',
 'DOSEUNIT',
 'DRT',
 'DRUG',
 'FREQ',
 'LABTEST',
 'NUM',
 'PROBLEM',
 'THERPROCDEV',
 'TK',
 'TOD',
 'TUNIT',
 'UNIT'}

Number of clinical concepts per consultation


{'1': 168,
 '2': 148,
 '3': 142,
 '4': 160,
 '5': 155,
 '6': 160,
 '7': 191,
 '8': 146}

How many times each classification appeared in each transcript


{'1': {'BIOSUB': 5,
  'BODYLOC': 14,
  'BODYMEAS': 15,
  'DDF': 0,
  'DIAGPROC': 10,
  'DISORDER': 41,
  'DOSEUNIT': 1,
  'DRT': 0,
  'DRUG': 10,
  'FREQ': 0,
  'LABTEST': 17,
  'NUM': 13,
  'PROBLEM': 112,
  'THERPROCDEV': 5,
  'TK': 57,
  'TOD': 1,
  'TUNIT': 2,
  'UNIT': 0},
 '2': {'BIOSUB': 12,
  'BODYLOC': 13,
  'BODYMEAS': 24,
  'DDF': 1,
  'DIAGPROC': 0,
  'DISORDER': 30,
  'DOSEUNIT': 4,
  'DRT': 0,
  'DRUG': 6,
  'FREQ': 0,
  'LABTEST': 9,
  'NUM': 9,
  'PROBLEM': 91,
  'THERPROCDEV': 11,
  'TK': 37,
  'TOD': 3,
  'TUNIT': 2,
  'UNIT': 2},
 '3': {'BIOSUB': 7,
  'BODYLOC': 11,
  'BODYMEAS': 11,
  'DDF': 0,
  'DIAGPROC': 0,
  'DISORDER': 35,
  'DOSEUNIT': 2,
  'DRT': 0,
  'DRUG': 9,
  'FREQ': 0,
  'LABTEST': 7,
  'NUM': 13,
  'PROBLEM': 91,
  'THERPROCDEV': 12,
  'TK': 37,
  'TOD': 0,
  'TUNIT': 2,
  'UNIT': 0},
 '4': {'BIOSUB': 3,
  'BODYLOC': 6,
  'BODYMEAS': 20,
  'DDF': 0,
  'DIAGPROC': 2,
  'DISORDER': 48,
  'DOSEUNIT': 4,
  'DRT': 0,
  'DRUG': 8,
  'FREQ': 0,
  'LABTEST': 

clin_concepts_set


{'aaa',
 'abuse',
 'ah',
 'alcohol',
 'all',
 'am',
 'an',
 'and',
 'antibiotics',
 'arms',
 'as',
 'ask',
 'asked',
 'asking',
 'at',
 'back',
 'bad',
 'banding',
 'be',
 'before',
 'blood',
 'bloods',
 'breath',
 'breathe',
 'breathing',
 'breaths',
 'but',
 'call',
 'can',
 'care',
 'catch',
 'cell',
 'chart',
 'chat',
 'chest',
 'chips',
 'coag',
 'cop',
 'cough',
 'coughing',
 'coughs',
 'cultures',
 'day',
 'days',
 'dhs',
 'did',
 'difficulty',
 'discomfort',
 'do',
 'does',
 'doing',
 'don',
 'door',
 'dose',
 'down',
 'dropped',
 'due',
 'e',
 'eat',
 'eating',
 'ecg',
 'eh',
 'eight',
 'eleven',
 'em',
 'emergency',
 'even',
 'ever',
 'examine',
 'fall',
 'fast',
 'feel',
 'feeling',
 'feels',
 'find',
 'finding',
 'five',
 'flu',
 'fluids',
 'food',
 'for',
 'four',
 'gaviscon',
 'get',
 'getting',
 'go',
 'going',
 'got',
 'gp',
 'h',
 'had',
 'hands',
 'hang',
 'hard',
 'he',
 'heart',
 'her',
 'hi',
 'history',
 'hm',
 'home',
 'hundred',
 'if',
 'indigestion',
 'initiall

In [12]:
xxx

classes = UMLS SEMANTIC TYPES.
all unique i.e. to find any class can just check if it is IN the list of classes.

NOTE: PROBLEM = problem/ findings in umls
    
WITH 3 EXCEPTIONS
unit
tunit
doseunit


SyntaxError: invalid syntax (<ipython-input-12-b5fcff92b3d0>, line 3)